In [3]:
# %%
# ============================================================
# Qrucible V3 - Quantum Module Analysis
# ============================================================
# VQE, QAOA, and Quantum Annealing Performance Evaluation
# ------------------------------------------------------------
# This script covers:
# - VQE accuracy validation
# - QAOA performance
# - Quantum Annealing simulation
# - Computational cost comparison
# - Generates figures (Fig 5 & Fig 6)
# ============================================================
!pip install numpy pandas matplotlib seaborn
# %%
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from qiskit import Aer, QuantumCircuit
from qiskit.algorithms import VQE, QAOA
from qiskit.algorithms.optimizers import COBYLA, SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import X, Z, I
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")

# %%
# ============================================================
# 1. VQE Ground State Energy Calculations
# ============================================================

def calculate_h2_energy_fci(bond_length):
    """Compute H2 ground state energy using Full Configuration Interaction (exact)."""
    molecule = Molecule(
        geometry=[['H', [0., 0., 0.]], ['H', [0., 0., bond_length]]],
        charge=0, multiplicity=1
    )
    driver = PySCFDriver.from_molecule(molecule, basis='sto3g')
    problem = ElectronicStructureProblem(driver)
    second_q_ops = problem.second_q_ops()
    main_op = second_q_ops[0]
    from qiskit.algorithms import NumPyMinimumEigensolver
    converter = QubitConverter(JordanWignerMapper())
    qubit_op = converter.convert(main_op)
    solver = NumPyMinimumEigensolver()
    result = solver.compute_minimum_eigenvalue(qubit_op)
    return result.eigenvalue.real


def calculate_h2_energy_vqe(bond_length):
    """Compute H2 ground state energy using VQE."""
    molecule = Molecule(
        geometry=[['H', [0., 0., 0.]], ['H', [0., 0., bond_length]]],
        charge=0, multiplicity=1
    )
    driver = PySCFDriver.from_molecule(molecule, basis='sto3g')
    problem = ElectronicStructureProblem(driver)
    second_q_ops = problem.second_q_ops()
    main_op = second_q_ops[0]
    converter = QubitConverter(JordanWignerMapper())
    qubit_op = converter.convert(main_op)
    ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', entanglement='linear', reps=3)
    optimizer = COBYLA(maxiter=1000)
    backend = Aer.get_backend('aer_simulator_statevector')
    vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=backend)
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return result.eigenvalue.real

print("✓ VQE calculation functions defined")

# %%
# Run VQE & FCI comparison for different bond lengths
bond_lengths = [0.5, 0.7, 1.0, 1.5]
fci_energies, vqe_energies = [], []

print("Calculating H₂ energies...\n")
print(f"{'Bond Length (Å)':<20}{'FCI Energy (Ha)':<20}{'VQE Energy (Ha)':<20}{'Error (%)':<10}")
print("="*70)

for bl in bond_lengths:
    fci = calculate_h2_energy_fci(bl)
    vqe = calculate_h2_energy_vqe(bl)
    err = abs((vqe - fci) / fci) * 100
    fci_energies.append(fci)
    vqe_energies.append(vqe)
    print(f"{bl:<20.2f}{fci:<20.4f}{vqe:<20.4f}{err:<10.3f}")

vqe_results = pd.DataFrame({
    'Bond_Length_Angstrom': bond_lengths,
    'FCI_Energy_Ha': fci_energies,
    'VQE_Energy_Ha': vqe_energies,
    'Error_Percent': [abs((v - f) / f) * 100 for v, f in zip(vqe_energies, fci_energies)]
})

print("\n✓ VQE validation complete")
print(f"Average Error: {vqe_results['Error_Percent'].mean():.4f}%")
print(f"Max Error: {vqe_results['Error_Percent'].max():.4f}%")

# %%
# ============================================================
# 2. Generate Figure 5: H₂ Potential Energy Curve
# ============================================================

extended_bond_lengths = np.linspace(0.4, 2.5, 20)
extended_fci = [calculate_h2_energy_fci(b) for b in extended_bond_lengths]
extended_vqe = [calculate_h2_energy_vqe(b) for b in extended_bond_lengths]

plt.figure(figsize=(10, 6))
plt.plot(extended_bond_lengths, extended_fci, 'b-', label='FCI (Exact)')
plt.plot(extended_bond_lengths, extended_vqe, 'r--', label='VQE')
plt.xlabel("Bond Length (Å)")
plt.ylabel("Ground State Energy (Hartree)")
plt.title("H₂ Potential Energy Curve: VQE vs FCI")
plt.legend()
plt.tight_layout()
plt.savefig("fig5_h2_potential_energy_curve.png", dpi=300)
print("✓ Figure 5 saved")
plt.show()

# %%
# ============================================================
# 3. QAOA Performance Analysis
# ============================================================

def qaoa_conformational_search(n_rotatable_bonds, p_layers=3):
    np.random.seed(42)
    torsion_energies = np.random.uniform(-2, 2, n_rotatable_bonds)
    interaction_energies = np.random.uniform(-1, 1, (n_rotatable_bonds, n_rotatable_bonds))
    H = sum([torsion_energies[i] * (Z ^ I ^ (n_rotatable_bonds - i - 1)) for i in range(n_rotatable_bonds)])
    for i in range(n_rotatable_bonds):
        for j in range(i+1, n_rotatable_bonds):
            H += interaction_energies[i,j] * ((Z ^ I ^ (n_rotatable_bonds - i - 1)) @ (Z ^ I ^ (n_rotatable_bonds - j - 1)))
    optimizer = COBYLA(maxiter=100)
    backend = Aer.get_backend('qasm_simulator')
    qaoa = QAOA(optimizer=optimizer, reps=p_layers, quantum_instance=backend)
    result = qaoa.compute_minimum_eigenvalue(H)
    classical_min = min([sum([torsion_energies[i] * (2*((conf >> i) & 1) - 1) for i in range(n_rotatable_bonds)]) for conf in range(2**n_rotatable_bonds)])
    return result.eigenvalue.real, classical_min, result.optimal_point

print("✓ QAOA function ready")

# %%
# Run QAOA for multiple bond counts
rotatable_bonds_range = [3, 4, 5, 6]
qaoa_results = []
for n in rotatable_bonds_range:
    qaoa_e, class_e, _ = qaoa_conformational_search(n)
    error_kcal = abs(qaoa_e - class_e) * 627.509
    qaoa_results.append([n, qaoa_e, class_e, error_kcal])
qaoa_df = pd.DataFrame(qaoa_results, columns=['n_rotatable_bonds', 'qaoa_energy', 'classical_energy', 'error_kcal_mol'])
print(qaoa_df)

# %%
# ============================================================
# 4. Quantum Annealing Simulation
# ============================================================

def simulated_quantum_annealing(n_fragments, n_iterations=500):
    np.random.seed(42)
    Q = np.random.uniform(-1, 1, (n_fragments, n_fragments))
    Q = (Q + Q.T) / 2
    state = np.random.randint(0, 2, n_fragments)
    best_state, best_E = state.copy(), state @ Q @ state
    for it in range(n_iterations):
        T = 10.0 * (1 - it / n_iterations) + 0.01
        flip = np.random.randint(n_fragments)
        new_state = state.copy(); new_state[flip] = 1 - new_state[flip]
        new_E = new_state @ Q @ new_state
        if new_E < best_E or np.random.rand() < np.exp(-(new_E - best_E)/T):
            state, best_E = new_state, new_E
            best_state = new_state.copy()
    return best_state, best_E

best_state, best_E = simulated_quantum_annealing(10)
print("✓ Quantum Annealing complete")
print("Best Energy:", best_E)

# %%
# ============================================================
# 5. Computational Cost Analysis
# ============================================================

cost_df = pd.DataFrame({
    'Approach': ['Pure Quantum (VQE)', 'Pure Classical (RF)', 'Hybrid Qrucible V3 (k=20)'],
    'Time_per_molecule_seconds': [7200, 0.001, None]
})

cost_df.loc[0, 'Total_time_hours'] = 7200*10000/3600
cost_df.loc[1, 'Total_time_hours'] = 0.001*10000/3600
cost_df.loc[2, 'Total_time_hours'] = (10/3600) + (7200*20/3600)
reduction = (1 - cost_df.loc[2, 'Total_time_hours'] / cost_df.loc[0, 'Total_time_hours']) * 100

print(cost_df)
print(f"\n🎯 COST REDUCTION: {reduction:.1f}% (Hybrid vs Pure Quantum)")

# %%
# ============================================================
# 6. Generate Figure 6: Computational Cost Comparison
# ============================================================

fig, ax = plt.subplots(figsize=(8,6))
approaches = ['Pure Quantum', 'Pure Classical', 'Hybrid Qrucible V3']
times = cost_df['Total_time_hours'].values
colors = ['#e74c3c', '#3498db', '#2ecc71']
ax.bar(approaches, times, color=colors, edgecolor='black')
ax.set_yscale('log')
ax.set_ylabel('Total Computation Time (hours, log scale)')
ax.set_title('Computational Cost for 10K Molecules')
plt.tight_layout()
plt.savefig('fig6_computational_cost.png', dpi=300)
print("✓ Figure 6 saved")
plt.show()


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Program Files\Python314\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'numpy'